In [28]:
import pandas as pd
import pymysql as sql

## 데이터베이스 접속하기

In [29]:
conn = sql.connect(
    host='localhost',
    user='root', 
    password='1234', 
    db='sakila', 
    charset='utf8',
)

## 데이터베이스 접근자 만든 후 해당 접근자로 데이터 가져오기

In [30]:
cur = conn.cursor()
cur.execute('select * from language')

6

In [31]:
rows = cur.fetchall()
rows

((1, 'English', datetime.datetime(2006, 2, 15, 5, 2, 19)),
 (2, 'Italian', datetime.datetime(2006, 2, 15, 5, 2, 19)),
 (3, 'Japanese', datetime.datetime(2006, 2, 15, 5, 2, 19)),
 (4, 'Mandarin', datetime.datetime(2006, 2, 15, 5, 2, 19)),
 (5, 'French', datetime.datetime(2006, 2, 15, 5, 2, 19)),
 (6, 'German', datetime.datetime(2006, 2, 15, 5, 2, 19)))

In [32]:
cur.description

(('language_id', 1, None, 3, 3, 0, False),
 ('name', 254, None, 60, 60, 0, False),
 ('last_update', 7, None, 19, 19, 0, False))

In [33]:
lang_df = pd.DataFrame(rows, columns=[i[0] for i in cur.description])
lang_df.set_index('language_id', inplace=True)
lang_df


,name,last_update
language_id,,
1,English,2006-02-15 05:02:19
2,Italian,2006-02-15 05:02:19
3,Japanese,2006-02-15 05:02:19
4,Mandarin,2006-02-15 05:02:19
5,French,2006-02-15 05:02:19
6,German,2006-02-15 05:02:19


In [39]:
# 복잡한 쿼리를 실행하기
query = """
select c.email
from customer as c
    inner join rental as r 
    on c.customer_id = r.customer_id
where date (r.rental_date) = (%s)
"""

cur.execute(query, ('2005-06-14'))
rows = cur.fetchall()
for row in rows:
    print(row[0])


JEFFERY.PINSON@sakilacustomer.org
ELMER.NOE@sakilacustomer.org
MINNIE.ROMERO@sakilacustomer.org
MIRIAM.MCKINNEY@sakilacustomer.org
DANIEL.CABRAL@sakilacustomer.org
TERRANCE.ROUSH@sakilacustomer.org
JOYCE.EDWARDS@sakilacustomer.org
GWENDOLYN.MAY@sakilacustomer.org
CATHERINE.CAMPBELL@sakilacustomer.org
MATTHEW.MAHAN@sakilacustomer.org
HERMAN.DEVORE@sakilacustomer.org
AMBER.DIXON@sakilacustomer.org
TERRENCE.GUNDERSON@sakilacustomer.org
SONIA.GREGORY@sakilacustomer.org
CHARLES.KOWALSKI@sakilacustomer.org
JEANETTE.GREENE@sakilacustomer.org


In [75]:
# 테이블을 생성 하여 적용하기
def create_table(conn, cur):
    try:
        beforequery = '''
        drop table if exists customer
        '''
        query = '''
        create table customer(
        name varchar(10),
        category smallint,
        region varchar(10))
        '''
        cur.execute(beforequery)
        cur.execute(query)
        conn.commit()
    except Exception as e:
        print(e)
#
# sqlclass_db 접속하기
connclass = sql.connect(
    host='localhost',
    user='root',
    password='1234',
    db='sqlclass_db',
    charset='utf8'
)

# 접근자 생성하기
curclass = connclass.cursor()
create_table(connclass, curclass)

query_addData ='''
insert into customer(name, category, region)
values ((%s), (%s), (%s))
'''
# 또는 (%s, %s, %s) 이렇게 하면 된다.
# curclass.execute(query_addData, ('김철수', 1, '서울'))
# curclass.execute(query_addData, ('이영희', 2, '경기'))
# curclass.execute(query_addData, ('박민수', 3, '부산'))
# curclass.execute(query_addData, ('최지혜', 4, '대구'))
# excutemany 를 통해서 한번에 값을 넣을수도 있다.
datas = [
    ('김철수', 1, '서울'),
    ('이영희', 2, '경기'),
    ('박민수', 3, '부산'),
    ('최지혜', 4, '대구'),
]
curclass.executemany(query_addData, datas)
connclass.commit()

curclass.execute('''select * from customer''')
result = curclass.fetchall()
pd.DataFrame(result)

updateQuery = '''
update customer
set region = '서울특별시'
where region = '서울'
'''
curclass.execute(updateQuery)

deleteQuery = '''
delete from customer
where name = %s
'''
curclass.execute(deleteQuery, '최지혜')
connclass.commit()


In [61]:

# curclass.close()
# connclass.close()

In [72]:
cur.close()
conn.close()

Error: Already closed